In [ ]:
pip install intel-tensorflow

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearnex import patch_sklearn
patch_sklearn()
import sklearn.datasets
from sklearn.model_selection import train_test_split

In [ ]:
# loading the data from sklearn
from sklearnex import patch_sklearn
patch_sklearn()
breast_cancer_dataset = sklearn.datasets.load_breast_cancer()

In [ ]:
print(breast_cancer_dataset)

In [ ]:
# loading the data to a data frame
data_frame = pd.DataFrame(breast_cancer_dataset.data, columns = breast_cancer_dataset.feature_names)
print(data_frame)

In [ ]:
# print the first 5 rows of the dataframe
data_frame.head()

In [ ]:
# adding the 'target' column to the data frame
data_frame['label'] = breast_cancer_dataset.target

In [ ]:
# print last 5 rows of the dataframe
data_frame.tail()

In [ ]:
# number of rows and columns in the dataset
data_frame.shape

In [ ]:
# getting some information about the data
data_frame.info()

In [ ]:
# checking for missing values
data_frame.isnull().sum()

In [ ]:
# statistical measures about the data
data_frame.describe()

In [ ]:
# checking the distribution of Target Varibale
data_frame['label'].value_counts()

1 --> Benign

0 --> Malignant

In [ ]:
data_frame.groupby('label').mean()

Separating the features and target

In [ ]:
X = data_frame.drop(columns='label', axis=1)
Y = data_frame['label']

In [ ]:
print(X)

In [ ]:
print(Y)

Splitting the data into training data & Testing data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

Standardize the data

In [ ]:
from sklearnex import patch_sklearn
patch_sklearn()
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

X_train_std = scaler.fit_transform(X_train)

X_test_std = scaler.transform(X_test)

**Building the Neural Network**

In [ ]:
# importing tensorflow and the oneDNN backend for Keras
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # set the logging level to ERROR
import tensorflow as tf 
tf.random.set_seed(3)
from tensorflow.python.keras import backend as K
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

In [ ]:
# configure the oneDNN backend
K.set_image_data_format('channels_last')
K.set_floatx('float32')
K.set_session(tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads=0,
    inter_op_parallelism_threads=0,
    allow_soft_placement=True,
    device_count={'CPU': 1})))

In [ ]:
# define the model using the oneDNN backend
model = tf.keras.Sequential([
    Rescaling(1./255, input_shape=(30,)),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dense(2, activation='sigmoid')
])
model.built = True

In [ ]:
# compiling the Neural Network

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# training the Meural Network

history = model.fit(X_train_std, Y_train, validation_split=0.1, epochs=100)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training data', 'validation data'], loc = 'lower right')

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')

plt.legend(['training data', 'validation data'], loc = 'upper right')

In [ ]:
loss, accuracy = model.evaluate(X_test_std, Y_test)
print(accuracy)

In [ ]:
print(X_test_std.shape)
print(X_test_std[0])

In [ ]:
Y_pred = model.predict(X_test_std)

In [ ]:
print(Y_pred.shape)
print(Y_pred[0])

In [ ]:
print(X_test_std)

In [ ]:
print(Y_pred)

In [ ]:
#  argmax function

my_list = [0.25, 0.56]

index_of_max_value = np.argmax(my_list)
print(my_list)
print(index_of_max_value)

In [ ]:
# converting the prediction probability to class labels

Y_pred_labels = [np.argmax(i) for i in Y_pred]
print(Y_pred_labels)

Building the predictive system

In [ ]:
input_data = (11.76,21.6,74.72,427.9,0.08637,0.04966,0.01657,0.01115,0.1495,0.05888,0.4062,1.21,2.635,28.47,0.005857,0.009758,0.01168,0.007445,0.02406,0.001769,12.98,25.72,82.98,516.5,0.1085,0.08615,0.05523,0.03715,0.2433,0.06563)

# change the input_data to a numpy array
input_data_as_numpy_array = np.asarray(input_data)

# reshape the numpy array as we are predicting for one data point
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

# standardizing the input data
input_data_std = scaler.transform(input_data_reshaped)

prediction = model.predict(input_data_std)
print(prediction)

prediction_label = [np.argmax(prediction)]
print(prediction_label)
print(prediction_label[0])
if(prediction_label[0] == 0):
  print('The tumor is Malignant')

else:
  print('The tumor is Benign')

In [ ]:
import pickle
import joblib


In [ ]:
#import bz2
#file = bz2.BZ2File('model2.pkl.h5', 'wb')
#pickle.dump(model, file)
#file.close()
joblib.dump(model, 'modeljob.pkl')


In [ ]:
model = joblib.load('modeljob.pkl')

In [ ]:
import pickle, bz2
from flask import Flask, render_template, request

In [ ]:
# Load the trained model
# with open('model1.pkl', 'rb') as f:
#     model = pickle.load(f)

#model = bz2.BZ2File('model1.pkl', 'rb')

# Initialize the Flask application
app = Flask(__name__)

# Define the route for the home page
@app.route('/')
def home():
    return render_template('home.html')

# Define the route for the prediction page
@app.route('/predict', methods=['POST'])
def predict():
    # Get the input values from the HTML form
    input_features = [float(x) for x in request.form.values()]

    # Make a prediction using the loaded model
    prediction = model.predict([input_features])[0]

    # Determine the output message based on the prediction
    if prediction == 0:
        output = 'Benign'
    else:
        output = 'Malignant'

    # Render the result template with the prediction
    return render_template('result.html', prediction=output)

# Run the Flask application
if __name__ == '__main__':
    app.run(debug=True, port=3004)


In [ ]:
# Initialize the Flask application
app = Flask(__name__)

In [ ]:
# Define the route for the home page
@app.route('/')
def home():
    return render_template('home.html')

In [ ]:
# Define the route for the prediction page
@app.route('/predict', methods=['POST'])
def predict():
    # Get the input values from the HTML form
    input_features = [float(x) for x in request.form.values()]

    # Make a prediction using the loaded model
    prediction = model.predict([input_features])[0]

    # Determine the output message based on the prediction
    if prediction == 0:
        output = 'Benign'
    else:
        output = 'Malignant'

    # Render the result template with the prediction
    return render_template('result.html', prediction=output)

In [ ]:
# Run the Flask application
if __name__ == '__main__':
    app.run(debug=True, port=3004)